# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines, thickness=2)
    return line_img, lines


# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def draw_lane_lines(img, lines):
    """
    Input:
    `img` is the output after applying hough and plotting points on blank image
    `lines' is list of points returned after hough transform
    
    Output:
    Blank image with left and right lane lines drawn on it
    
    Logic to extrapolate left and right lanes from set of points is as follows:
    1. Seperate left and right lane points based on slope
    2. Use cv2 fitLine method to get unit vector co-ordinates and a point(x,y) on the line
    3. fitline return [vx,vy,x,y], where vx, vy are the unit vector coordinates and point (x,y) on line
    4. compute slope and intercept using following logic
        slope = vy / vx
        intercept = y - (slope * x)
    5. compute x-axix points (x1,x2) from any fixed (y1,y2).
    6. Draw line using above computed (x1,y1) and (x2,y2)
    """
    #Iterate over points
    left_points = []
    right_points = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1) 
            if slope < -0.4 and slope > -0.9: #left lane points
                left_points.append((x1,y1))
                left_points.append((x2,y2))
            elif slope > 0.4 and slope < 0.9: #right lane points
                right_points.append((x1,y1))
                right_points.append((x2,y2))
                
    #Left lane line. Determine co-oridinates((x1,y1) and (x2,y2)) to draw line 
    [left_vx,left_vy,left_x,left_y] = cv2.fitLine(np.array(left_points, dtype=np.int32), cv2.DIST_L2,0,0.01,0.01)
    left_slope = left_vy / left_vx
    left_intercept = left_y - (left_slope * left_x)
    left_startY = img.shape[0] #start from bottom
    left_endY = int(img.shape[0] * 0.60) #upto 66% of image
    left_startX = (left_startY - left_intercept) / left_slope
    left_endX = (left_endY - left_intercept) / left_slope
    
    #Right lane line. Determine co-oridinates((x1,y1) and (x2,y2)) to draw line
    [right_vx,right_vy,right_x,right_y] = cv2.fitLine(np.array(right_points, dtype=np.int32), cv2.DIST_L2,0,0.01,0.01)
    right_slope = right_vy / right_vx
    right_intercept = right_y - (right_slope * right_x)
    right_startY = img.shape[0]
    right_endY = int(img.shape[0] * 0.60)
    right_startX = (right_startY - right_intercept) / right_slope
    right_endX = (right_endY - right_intercept) / right_slope
    
    #empty image
    laneline_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    #draw lane lines on image
    cv2.line(laneline_img, (left_startX,left_startY), (left_endX,left_endY), (255,0,0), 10)
    cv2.line(laneline_img, (right_startX,right_startY), (right_endX,right_endY), (255,0,0), 10)
    
    return laneline_img


In [ ]:

def fetch_lines(image):
    """
    Input: `img` raw image
    Output: Set of points returned by hough_lines method.  List[(x1,y1,x2,y2),(x1,y1,x2,y2)]
    
    fetch_lines method does following steps before applying hough transform on the image:
    1. Grayscale
    2. Gaussian smoothing (to remove noise)
    3. Get list of edges by applying canny
    4. Select region of interest
    5. Apply hough transform and return points list
    """
    #1. Grayscale
    image_copy = grayscale(image)
    
    #2. Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussian_blur(image_copy,kernel_size)
  
    #3. Canny edge detection
    low_threshold = 20
    high_threshold = 200
    edges = canny(blur_gray, low_threshold, high_threshold)
    
    #4. Region of interest. Mask the edges
    imshape = image.shape
    vertices = np.array([[(110,imshape[0]),(imshape[1]/2-25, imshape[0]/2+50),
                          (imshape[1]/2+25, imshape[0]/2+50), (imshape[1]-20,imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    
    #5. Hough transform
    # Define the Hough transform parameters
    rho = 2
    theta = np.pi/180
    threshold = 15
    min_line_length = 2
    max_line_gap = 2

    line_img, lines = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
    
    return lines, line_img, masked_edges

def pipeline(image):
    """
    pipeline method takes input as image and return image after drawing lane lines.
    Input: `image` on which we need to detect left and right lane lines
    Output: final image after drawing both left and right lane lines
    
    Following is logic for drawing lines on image:
    1. Apply hough_lines and get set of points
    2. Take points list returned in step#1 and extrapolate and find left and right lane lines
    3. return final image
    """
    #Get all lines and line image
    lines, line_img, masked_edges = fetch_lines(image)
    
    #Extrapolate all points and draw lane lines (left,right)
    lane_lines = draw_lane_lines(masked_edges, lines)
    
    #final image with updated lane lines(left,right)
    final_image = weighted_img(lane_lines, image, α=0.8)
    
    return final_image

In [ ]:
import os
images = os.listdir("test_images/")

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:

#Iterate over all test images and test pipeline
def run():
     
    i=1
    for image in images:
        #image object
        original_image = mpimg.imread('test_images/'+image)
    
        #get image with left and right lanes plotted 
        final_image = pipeline(original_image)
        plt.subplot(3,2,i) #display inline
        plt.imshow(final_image)
      
        mpimg.imsave('test_images/'+'output_'+image, final_image) #save final image
        i = i+1
    
run()

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
       
    #return final image
    final_image = pipeline(image)

    return final_image

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

My pipeline has following steps:
    1. Grayscale
    2. Gaussian smoothing (to remove noise)
    3. Get list of edges by applying canny
    4. Select region of interest
    5. Apply hough transform and return points list
    6. Draw left and right lanes using points given by Hough
    
    Logic to extrapolate left and right lanes from set of points is as follows:
    
    a. Seperate left and right lane points based on slope
    b. Use cv2 fitLine method to get unit vector co-ordinates and a point(x,y) on the line
    c. fitline return [vx,vy,x,y], where vx, vy are the unit vector coordinates and point (x,y) on line
    d. compute slope and intercept using following logic
        slope = vy / vx
        intercept = y - (slope * x)
    e. compute x-axix points (x1,x2) from any fixed (y1,y2).
    f. Draw line using above computed (x1,y1) and (x2,y2)

A more robust solution would probably use

    a. Tweaked parameters for the hough transformation
    b. a non-linear fitting with higher degree for detecting curves than straight lines
    c. Apply some smoothing logic for better transition between frames
    d. deep learning model



## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))